In [13]:
import pandas as pd

df = pd.read_csv("./data/maine.csv")
rank_cols = [col for col in df.columns if "Choice" in col]
df = df[rank_cols]

def clean_name(name):
    if pd.isna(name) or name.strip() == "":
        return None
    name = name.strip()
    if name.lower() in ["undervote", "overvote"]:
        return name.lower()
    return name.split(",")[0].split()[-1]

df = df.applymap(clean_name)
df = df.dropna(how="all")
unique_candidates = sorted(set(
    cell for cell in df.values.flatten()
    if pd.notna(cell) and cell.strip() != ""
))
ballots = df.apply(lambda row: ",".join(cell for cell in row if pd.notna(cell) and cell.strip() != ""), axis=1)

with open("./data/converted_ballots.csv", "w", newline="\n") as f:
    f.write(",".join(unique_candidates) + "\n")
    for ballot in ballots:
        f.write(ballot + "\n")

C:\Users\noddl\AppData\Local\Temp\ipykernel_767992\3372463343.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_name)


In [14]:
import csv
from collections import defaultdict
import pandas as pd

# Load ballots
with open("data/converted_ballots.csv") as f:
    reader = csv.reader(f)
    ballots = [row for row in reader if row]

# Define what counts as not a valid vote
non_candidates = {'undervote', 'overvote', ''}

# Identify all candidates
candidates = sorted(set(name for ballot in ballots for name in ballot if name not in non_candidates))

# Count how many times each candidate is *not* ranked in a ballot that has at least one valid vote
undervote_counts = defaultdict(int)
total_valid_ballots = 0

for ballot in ballots:
    ranked = [name for name in ballot if name not in non_candidates]
    if ranked:
        total_valid_ballots += 1
        ranked_set = set(ranked)
        for candidate in candidates:
            if candidate not in ranked_set:
                undervote_counts[candidate] += 1

# Create a dataframe
df = pd.DataFrame([
    {
        "Candidate": candidate,
        "Times_Not_Ranked": undervote_counts[candidate],
        "Percent_Of_Valid_Ballots": round(undervote_counts[candidate] / total_valid_ballots * 100, 2)
    }
    for candidate in candidates
])

# Sort and save
df.sort_values(by="Percent_Of_Valid_Ballots", ascending=False, inplace=True)
df.to_csv("candidate_undervote_analysis.csv", index=False)

print(df)


  Candidate  Times_Not_Ranked  Percent_Of_Valid_Ballots
2      Hoar            170395                     58.73
0      Bond            158547                     54.64
1    Golden            109692                     37.81
3  Poliquin            100522                     34.64


In [15]:
import csv
from collections import defaultdict
import pandas as pd
from itertools import combinations

# Load ballots
with open("data/converted_ballots.csv") as f:
    reader = csv.reader(f)
    ballots = [row for row in reader if row]

# Define non-candidates
non_candidates = {'undervote', 'overvote', ''}

# Get all candidates
candidates = sorted(set(name for ballot in ballots for name in ballot if name not in non_candidates))

# All non-empty subsets (excluding full set)
subset_list = []
for r in range(1, len(candidates)):
    subset_list.extend(combinations(candidates, r))

# Prepare counting
subset_counts = defaultdict(int)
valid_ballots = 0

# Count how many ballots exclude each subset
for ballot in ballots:
    ranked = [name for name in ballot if name not in non_candidates]
    if ranked:
        valid_ballots += 1
        ranked_set = set(ranked)
        for subset in subset_list:
            if not any(c in ranked_set for c in subset):
                subset_counts[subset] += 1

# Convert to DataFrame
rows = []
for subset, count in subset_counts.items():
    rows.append({
        "Undervoted_Candidates": ", ".join(subset),
        "Times_Not_Ranked": count,
        "Percent_Of_Valid_Ballots": round(count / valid_ballots * 100, 2)
    })

df = pd.DataFrame(rows)
df.sort_values(by="Percent_Of_Valid_Ballots", ascending=False, inplace=True)

# Save to CSV
df.to_csv("candidate_undervote_subsets.csv", index=False)

print(df.head(10))  # Display top 10 most frequent subsets


  Undervoted_Candidates  Times_Not_Ranked  Percent_Of_Valid_Ballots
2                  Hoar            170395                     58.73
0                  Bond            158547                     54.64
4            Bond, Hoar            148232                     51.09
1                Golden            109692                     37.81
7              Poliquin            100522                     34.64
5          Golden, Hoar             97336                     33.55
3          Bond, Golden             94921                     32.71
6    Bond, Golden, Hoar             89669                     30.90
8        Hoar, Poliquin             67710                     23.34
9        Bond, Poliquin             57394                     19.78


In [ ]:
import pandas as pd

# Load your CSV with the subset summary
df = pd.read_csv("candidate_undervote_subsets.csv")  # or whatever your filename is

# Clean up column if needed
df['Undervoted_Candidates'] = df['Undervoted_Candidates'].str.strip()

# Get unique list of all candidates in the dataset
all_candidates = sorted({
    cand.strip()
    for subset in df['Undervoted_Candidates']
    for cand in subset.split(',')
})

# Expand to matrix format
rows = []
for _, row in df.iterrows():
    subset = [c.strip() for c in row['Undervoted_Candidates'].split(',')]
    for candidate in all_candidates:
        rows.append({
            'Subset_Label': row['Undervoted_Candidates'],
            'Candidate': candidate,
            'In_Subset': 1 if candidate in subset else 0,
            'Times_Not_Ranked': row['Times_Not_Ranked']
        })

# Output to CSV
output_df = pd.DataFrame(rows)
output_df.to_csv("undervote_matrix_for_tableau.csv", index=False)


✅ Matrix format saved to 'undervote_matrix_for_tableau.csv'


In [ ]:
m = [1, 2, 3, 4, 5,]
print(m[-3])


3


In [16]:
import csv
import itertools
from collections import defaultdict
import pandas as pd
from itertools import combinations
# Load the ballots from the CSV
with open("data/converted_ballots.csv") as f:
    reader = csv.reader(f)
    ballots = [row for row in reader if row]

# Define non-candidates
non_candidates = {'overvote', 'undervote', ''}
candidates = sorted(set(name for ballot in ballots for name in ballot if name not in non_candidates))

# Initialize margin of victory matrix
pairwise_margins = defaultdict(lambda: defaultdict(int))

# For each pair of candidates
for c1, c2 in combinations(candidates, 2):
    c1_wins = 0
    c2_wins = 0
    for ballot in ballots:
        # Keep only c1 and c2, in original order
        filtered = [cand for cand in ballot if cand in (c1, c2)]
        if len(filtered) < 2:
            continue  # skip if both candidates not present
        if filtered[0] == c1:
            c1_wins += 1
        else:
            c2_wins += 1
    # Record margin (c1 over c2) and the inverse (c2 over c1)
    margin = c1_wins - c2_wins
    pairwise_margins[c1][c2] = margin
    pairwise_margins[c2][c1] = -margin  # inverse margin

# Convert to DataFrame for export
df = pd.DataFrame(pairwise_margins).fillna(0).astype(int)
df.index.name = 'Candidate vs'
df.to_csv("pairwise_margins.csv")

In [17]:
import pandas as pd

df = pd.read_csv("./data/maine.csv")
first_choice_col = "Rep. to Congress 1st Choice District 2"
first_choices = df[first_choice_col].fillna("undervote").str.strip()
total_valid = first_choices[first_choices.isin(["REP Poliquin, Bruce", "DEM Golden, Jared F."])].count()
rep_count = (first_choices == "REP Poliquin, Bruce").sum()
dem_count = (first_choices == "DEM Golden, Jared F.").sum()
rep_percent = 100 * rep_count / total_valid if total_valid else 0
dem_percent = 100 * dem_count / total_valid if total_valid else 0

print(f"Republican First-Choice: {rep_percent:.1f}%")
print(f"Democrat First-Choice:   {dem_percent:.1f}%")

Republican First-Choice: 50.4%
Democrat First-Choice:   49.6%


In [18]:
def pivot_method_outcomes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts wide-form method outcomes (Method, Candidate1, Score1, ...) 
    to long-form (Method, Candidate, Score).
    """
    long_rows = []
    for _, row in df.iterrows():
        method = row['Method']
        for j in range(1, len(row), 2):
            candidate = row[j]
            score = row[j+1] if j+1 < len(row) else None
            if pd.notna(candidate) and pd.notna(score):
                long_rows.append({
                    "Method": method,
                    "Candidate": candidate,
                    "Score": float(score)
                })
    return pd.DataFrame(long_rows)

import pandas as pd

def normalize_method_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize scores per method (row-wise), for either:
    - wide-form: Method, Candidate1, Score1, Candidate2, Score2, ...
    - long-form: Method, Candidate, Score

    Returns: long-form DataFrame with columns:
        Method, Candidate, Score, NormalizedScore
    """
    if 'Candidate' in df.columns and 'Score' in df.columns:
        # Long form: normalize directly
        def normalize_group(group):
            min_score = group['Score'].min()
            max_score = group['Score'].max()
            range_score = max_score - min_score if max_score != min_score else 1.0
            group['NormalizedScore'] = (group['Score'] - min_score) / range_score
            return group

        return df.groupby('Method', group_keys=False).apply(normalize_group)

    elif 'Method' in df.columns:
        # Wide form: pivot first
        long_rows = []
        for _, row in df.iterrows():
            method = row['Method']
            candidates = []
            scores = []
            for j in range(1, len(row), 2):
                candidate = row[j]
                score = row[j+1] if j+1 < len(row) else None
                if pd.notna(candidate) and pd.notna(score):
                    candidates.append(candidate)
                    scores.append(float(score))
            if not scores:
                continue
            min_score = min(scores)
            max_score = max(scores)
            range_score = max_score - min_score if max_score != min_score else 1.0
            for candidate, score in zip(candidates, scores):
                normalized_score = (score - min_score) / range_score
                long_rows.append({
                    "Method": method,
                    "Candidate": candidate,
                    "Score": score,
                    "NormalizedScore": normalized_score
                })
        return pd.DataFrame(long_rows)

    else:
        raise ValueError("Unrecognized format. Must contain 'Method' column and either wide candidate/score pairs or long-form 'Candidate' and 'Score'.")


def write_method_outcomes(df: pd.DataFrame, path: str) -> None:
    """
    Writes a DataFrame to the given CSV file path.
    """
    df.to_csv(path, index=False)

import pandas as pd
wide_df = pd.read_csv("./data/method_outcomes.csv")

# Get pivoted long-form
#pivoted_df = pivot_method_outcomes(wide_df)

# Normalize the long-form data
normalized_df = normalize_method_scores(wide_df)

# Save results
#write_method_outcomes(w, "./data/method_outcomes_long.csv")
write_method_outcomes(normalized_df, "./data/method_outcomes_normalized2.csv")

C:\Users\noddl\AppData\Local\Temp\ipykernel_767992\3479190025.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  candidate = row[j]
C:\Users\noddl\AppData\Local\Temp\ipykernel_767992\3479190025.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  score = row[j+1] if j+1 < len(row) else None


In [19]:
import pandas as pd

# Load your raw data
df = pd.read_csv("data/converted_ballots.csv", header=None)

# Rename columns based on rank
df.columns = ['Rank_1', 'Rank_2', 'Rank_3', 'Rank_4', 'Rank_5', 'Rank_6']

# Add Ballot ID (starting from 1)
df['Ballot_ID'] = df.index + 1

# Melt to long format
df_long = df.melt(id_vars='Ballot_ID', 
                  var_name='Rank', 
                  value_name='Candidate')

# Convert Rank_1 → 1, Rank_2 → 2, ...
df_long['Rank'] = df_long['Rank'].str.extract('(\d)').astype(int)

# Optional: filter out undervotes/overvotes
df_cleaned = df_long[~df_long['Candidate'].isin(['undervote', 'overvote'])]

# Save to CSV for Tableau
df_cleaned.to_csv("ballot_cleaned_for_tableau.csv", index=False)


<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
C:\Users\noddl\AppData\Local\Temp\ipykernel_767992\3557418131.py:18: SyntaxWarning: invalid escape sequence '\d'
  df_long['Rank'] = df_long['Rank'].str.extract('(\d)').astype(int)
C:\Users\noddl\AppData\Local\Temp\ipykernel_767992\3557418131.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/converted_ballots.csv", header=None)
